In [1]:
#PREPROCESS_identify_cytokines_from_significant_proteins

import pandas as pd

In [2]:
def get_list_from_file(data_file):
    
    data_df = pd.read_csv(data_file, sep ="\t", header=None)
    feature_list = []
    r, c = data_df.shape
    
    for i in range(r):
        value = data_df.iloc[i][0]
        feature_list.append(value)
        
    return feature_list


def get_cytokine_list_from_file(data_file):
    
    data_df = pd.read_csv(data_file, sep ="\t")
    feature_list = []
    r, c = data_df.shape
    
    for i in range(r):
        value = data_df.iloc[i][1]
        feature_list.append(value)
        
    return feature_list

def find_significant_cytokine(data_dict, cytokine_list):
    
    all_cytokine_list = []
    for data_type in list(data_dict.keys()):
        significant_protein_list = data_dict[data_type]
        temp_cytokine_list = []
        print (data_type)

        for protein_symbol_with_barcode in significant_protein_list:
            protein_symbol = protein_symbol_with_barcode.split('_')[0]
            if protein_symbol in cytokine_list:
                temp_cytokine_list.append(protein_symbol_with_barcode)
                all_cytokine_list.append(protein_symbol_with_barcode)
        
        print ("%s: %s cytokines" % (data_type, len(temp_cytokine_list)))
        print (temp_cytokine_list)
        
    all_cytokine_list = list(set(all_cytokine_list))
    return (all_cytokine_list)
    

In [3]:
cytokine_file = '../../../etc/Cytokines.txt'
cytokine_list = get_cytokine_list_from_file(cytokine_file)

In [4]:
data_dir = '../../../analysis/statistics/gse/proteomics'
condition_list = ["cVSneg","cVSpos"]
abundance_type_list = ["up","down"]

data_dict = {}
for abundance_type in abundance_type_list:
    for condition in condition_list:
        data_file = '%s/proteomics.diff.%s.%s.barcode.tsv' % (data_dir, condition, abundance_type)
        feature_list = get_list_from_file(data_file)  
        
        data_type = "%s_%s" %(condition, abundance_type)
        data_dict[data_type] = feature_list

all_cytokine_list = find_significant_cytokine(data_dict, cytokine_list)

cVSneg_up
cVSneg_up: 2 cytokines
['CCL7_22969-12', 'C5_2381-52']
cVSpos_up
cVSpos_up: 1 cytokines
['CXCL13_3487-32']
cVSneg_down
cVSneg_down: 3 cytokines
['SECTM1_13093-6', 'HDGFL3_18899-82', 'NAMPT_5011-11']
cVSpos_down
cVSpos_down: 5 cytokines
['SECTM1_13093-6', 'HDGFL3_18899-82', 'FGF12_4392-54', 'NAMPT_5011-11', 'EDN1_6495-14']


In [78]:
proteomic_file = '../../../preprocessed_data/proteomics/somascan_anml.T.v2.tsv'
protein_df = pd.read_csv(proteomic_file, sep="\t", index_col=0)
subset_protein_df = protein_df.loc[all_cytokine_list]

patient_info_file = '../../../preprocessed_data/meta/patient_info.ML_ready.tsv'
patient_df = pd.read_csv(patient_info_file, sep="\t",index_col=0)
subset_protein_df = subset_protein_df.append(patient_df.loc["acpa"])
subset_protein_df.to_csv('../../../analysis/statistics/cytokine_boxplots/8_cytokine_df.tsv', sep="\t")